### Object tracking with opencv

In [1]:
import cv2

In [9]:
def initialize(videopath):
    cap = cv2.VideoCapture(videopath)
    od = cv2.createBackgroundSubtractorMOG2(varThreshold=1000, history=20, detectShadows=False) 
    return cap, od

#### custom tracking with ROI

In [12]:
cap, od =initialize('../data/aerial_tennis_video.mp4')
ret, frame_n = cap.read()  # Read the frame
bbox=cv2.selectROI(frame_n)
tracker=cv2.legacy.TrackerMOSSE_create()
# tracker = cv2.legacy.TrackerTLD_create()
ok = tracker.init(frame_n ,bbox)
while True:
    ret, frame = cap.read() 
    if not ret:
        break
    ok, bbox = tracker.update(frame)
    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
    else :
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):                                         # exit
        break        
    elif key == ord('x'):                                       # pauze screen
        cv2.waitKey(0)            

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


[-1, -1, -1, -1]

### OLD

In [ ]:
cap, od =initialize('../data/aerial_tennis_video.mp4')
ret, frame_n = cap.read()  # Read the frame
bbox=cv2.selectROI(frame_n)
tracker=cv2.legacy.TrackerMOSSE_create()
ok = tracker.init(frame_n ,bbox)
while True:
    ret, frame = cap.read() 
    mask = od.apply(frame)                                      # run mask (black and white)
    _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contrasting values in mask
    cv2.imshow('mask', mask)    
    if not ret:
        break
    ok, bbox = tracker.update(frame)
    if ok:
        print('tracking!')
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
    else :
        print('not tracked')
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):                                         # exit
        break        
    elif key == ord('x'):                                       # pauze screen
        cv2.waitKey(0)            

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


In [13]:
## first step was finding objects, now its time for tracking

## to do , code for tracking a specific box > make a working function
## setup first boxes based on first setup and iterate over all boxes

In [16]:
def cleanup_frame(frame, od):
    # frame = cv2.resize(frame, (960, 540))                             # Resize image
    mask = od.apply(frame)                                              # run mask (black and white)
    _, extra_mask = cv2.threshold(mask, 250, 255, cv2.THRESH_BINARY)    # set treshold to only get really contrasting values in mask
    return extra_mask

In [17]:
def identifying_contours_players(c, od):
    count = 0
    frame_nr = 6
    while True:
        count +=1
        print(c.get(1))     
        ret, frame = c.read()                                       # read frames
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        # frame = cv2.resize(frame, (960, 540))                       # Resize image
        # cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contrasting values in mask
        cv2.imshow('mask', mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        frame_detect = []
        if count == frame_nr:
            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
                frame_detect.append([x,y,w,h])
                cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
                cv2.imshow('frame2', frame)  
                cv2.imwrite('test.png', frame)  
            cv2.waitKey(0)      
            print(frame_detect)
            break
            # break
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return frame_detect

In [18]:
# cap, od =initialize('../data/aerial_football_video.mp4')
cap, od =initialize('../data/aerial_tennis_video.mp4')
frame_bbox = identifying_contours_players(cap, od)

0.0
1.0
2.0
3.0
4.0
5.0
[[974, 923, 1, 1], [973, 915, 1, 7], [973, 864, 1, 49], [973, 861, 1, 1], [924, 860, 4, 1], [911, 860, 12, 1], [1106, 859, 14, 1], [1075, 859, 5, 1], [1061, 859, 1, 1], [973, 859, 1, 1], [720, 857, 14, 1], [1186, 855, 6, 1], [1407, 854, 1, 1], [1401, 854, 2, 1], [1344, 854, 7, 1], [1329, 854, 7, 1], [1270, 854, 7, 1], [753, 780, 18, 1], [736, 780, 8, 1], [1246, 777, 1, 1], [639, 777, 15, 1], [626, 777, 1, 1], [610, 777, 4, 1], [974, 776, 1, 3], [1517, 775, 1, 1], [983, 775, 3, 1], [977, 775, 4, 1], [957, 775, 1, 1], [954, 775, 1, 1], [1080, 774, 2, 1], [972, 766, 1, 56], [283, 547, 1, 1], [1348, 542, 1, 2], [705, 540, 1, 1], [699, 540, 2, 1], [834, 539, 6, 1], [832, 539, 1, 1], [822, 539, 7, 1], [819, 539, 2, 1], [1348, 538, 1, 3], [975, 538, 1, 1], [969, 538, 1, 6], [944, 538, 23, 1], [1075, 537, 6, 1], [1073, 537, 1, 1], [1340, 536, 3, 1], [1328, 536, 11, 1], [1317, 536, 10, 1], [972, 536, 1, 1], [970, 536, 1, 1], [900, 535, 7, 1], [1029, 534, 1, 1], [1023, 53

In [19]:
len(frame_bbox)

105

In [20]:
def identifying_objects(c, od):
    count = 0
    frame_nr = 6
    while True:
        count +=1
        print(c.get(1))     
        ret, frame = c.read()                                       # read frames
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        # frame = cv2.resize(frame, (960, 540))                       # Resize image
        # cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contrasting values in mask
        cv2.imshow('mask', mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        frame_detect = []
        if count == frame_nr:
            for cnt in contours:
                print('getting contours')
                x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
                frame_detect.append([x,y,w,h])
                cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
                cv2.imshow('frame2', frame)    
            # break
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return frame_detect

In [21]:
cap, od =initialize('../data/aerial_football_video.mp4')

frame_bbox = identifying_objects(cap, od)

0.0
1.0
2.0
3.0
4.0
5.0
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0


In [12]:
frame_bbox

[]

In [50]:
frame_bbox[:5]

[[837, 638, 3, 2],
 [837, 636, 1, 1],
 [835, 635, 1, 3],
 [837, 633, 1, 1],
 [695, 543, 2, 2]]

In [51]:
frame_bbox_large = []
for bb in frame_bbox:
    widthlarge = bb[2] * 50
    heightlarge = bb[3] * 50
    frame_bbox_large.append((bb[0], bb[1], widthlarge, heightlarge))

In [52]:
frame_bbox_large[:5]

[(837, 638, 150, 100),
 (837, 636, 50, 50),
 (835, 635, 50, 150),
 (837, 633, 50, 50),
 (695, 543, 100, 100)]

In [22]:
cap, od =initialize('../data/aerial_tennis_video.mp4')
ret, frame_n = cap.read()  # Read the frame

for start_contour in frame_bbox_large:
    bbox = start_contour
    # bbox = (546, 109, 150, 150)
    tracker=cv2.legacy.TrackerMOSSE_create()
    ok = tracker.init(frame_n ,bbox)
    first = None
    while True:
        if not first:
            cap.set(1,n+1) #otherwise starts  reading from 22 when starting with 20
            first ='done'
        ret, frame = cap.read() 
        # frame = cleanup_frame(frame, od)
        # print(cap.get(1))
        if not ret:
            break
        ok, bbox = tracker.update(frame)
        if ok:
            print('tracking!')
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
        else :
            print('not tracked')
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
        cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
        cv2.imshow("Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)            

    # cap.release()
    # cv2.destroyAllWindows()
    # [cv2.waitKey(1) for i in range(1,5)]           


NameError: name 'frame_bbox_large' is not defined

In [26]:
cap, od =initialize('../data/aerial_tennis_video.mp4')
n=20
framenr = cap.get(1)
print(framenr)
cap.set(1,n)  # Where frame_no is the frame you want
ret, frame_n = cap.read()  # Read the frame
# succes, first_frame = cap.read()
# frame100 = cleanup_frame(frame100, od)
bbox=cv2.selectROI(frame_n)
# bbox = (424, 278, 54, 32)  # test case
# bbox = (837, 638, 3, 2)
# bbox = (545, 112, 50, 50)
# bbox = (546, 109, 150, 150)
print(bbox)
# bbox = first_contours(first_frame)
tracker=cv2.legacy.TrackerMOSSE_create()
ok = tracker.init(frame_n ,bbox)
first = None
while True:
    if not first:
        cap.set(1,n+1) #otherwise starts  reading from 22 when starting with 20
        first ='done'
    ret, frame = cap.read() 
    # frame = cleanup_frame(frame, od)
    print(cap.get(1))
    if not ret:
        break
    ok, bbox = tracker.update(frame)
    print(ok)
    if ok:
        print('tracking!')
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
    else :
        print('not tracked')
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):                                         # exit
        break        
    elif key == ord('x'):                                       # pauze screen
        cv2.waitKey(0)            

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


0.0
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
(298, 616, 79, 76)
22.0
True
tracking!
23.0
True
tracking!
24.0
True
tracking!
25.0
True
tracking!
26.0
True
tracking!
27.0
True
tracking!
28.0
True
tracking!
29.0
True
tracking!
30.0
True
tracking!
31.0
True
tracking!
32.0
True
tracking!
33.0
True
tracking!
34.0
True
tracking!
35.0
True
tracking!
36.0
True
tracking!
37.0
True
tracking!
38.0
True
tracking!
39.0
True
tracking!
40.0
True
tracking!
41.0
True
tracking!
42.0
True
tracking!
43.0
True
tracking!
44.0
True
tracking!
45.0
True
tracking!
46.0
True
tracking!
47.0
True
tracking!
48.0
True
tracking!
49.0
True
tracking!
50.0
True
tracking!
51.0
True
tracking!
52.0
True
tracking!
53.0
True
tracking!
54.0
True
tracking!
55.0
True
tracking!
56.0
True
tracking!
57.0
True
tracking!
58.0
True
tracking!
59.0
True
tracking!
60.0
True
tracking!
61.0
True
tracking!
62.0
True
tracking!
63.0
True
tracking!
64.0
True
tracking!
65.0
True
track

[-1, -1, -1, -1]

In [6]:
#(424, 278, 54, 32) frame 20 starting point tracking bbox